# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [15]:
TRIAL_LABELS_DF = pd.read_excel("./proc/rce2_finalized_trial_labels.xlsx")

In [16]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_04_spectral_and_sleap.pkl")

In [17]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [18]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [19]:
BIN_TO_VELOCITY = {0: "0 to 2.5cm/s", 1: "2.5 to 5cm/s", 2: "5 to 10 cm/s", 3: "10cm/s+"}

In [20]:
ALL_BANDS = ["theta", "beta", "gamma"]
BAND_TO_FREQ = {"theta": {"low_freq": 6, "high_freq": 11}, "beta": {"low_freq": 20, "high_freq": 31}, "gamma": {"low_freq": 30, "high_freq": 51}}
BAND_TO_FREQ = {"theta": (6,11), "beta": (20,31), "gamma": (30,51)}

In [21]:
GROUPINGS = "trial_outcome"
# GROUPINGS = "competition_closeness"

In [22]:
FONTSIZE=20

In [23]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [24]:
def generate_pairs(lst):
    pairs = []
    n = len(lst)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((lst[i], lst[j]))
    return pairs

In [25]:
def nested_dict():
    return defaultdict(dict)

In [26]:
def mean_of_counter(counter):
    # Extract values from the Counter and calculate the mean
    values = list(counter.values())
    return sum(values) / len(values) if values else 0  # Prevent division by zero

In [27]:
def adjust_lightness(color, amount=0.5):
    """
    Adjusts the lightness of the given color by the provided amount.
    :param color: Input color in some format that matplotlib's `to_rgb` can handle.
    :param amount: Amount to adjust. > 1 for lighter, < 1 for darker.
    :return: Adjusted color.
    """
    import colorsys
    c = mcolors.to_rgb(color)
    h, l, s = colorsys.rgb_to_hls(*c)
    return colorsys.hls_to_rgb(h, max(0, min(1, amount * l)), s)

In [28]:
def generate_gradient_colors(base_color, num_colors=10):
    """
    Generate gradient colors based on a base color.
    
    :param base_color: The base color to create a gradient from.
    :param num_colors: The number of gradient colors to generate.
    :return: List of colors.
    """
    # Convert base color to HLS (Hue, Lightness, Saturation)
    h, l, s = colorsys.rgb_to_hls(*mcolors.to_rgb(base_color))

    # Determine the step size for lightness
    middle_index = num_colors // 2
    lightness_step = l / (middle_index if middle_index > 0 else 1)

    colors = []
    for i in range(num_colors):
        if i < middle_index:
            # For the lighter colors (before the middle)
            new_lightness = l + (middle_index - i) * lightness_step
        elif i == middle_index:
            # The middle color remains the base color
            new_lightness = l
        else:
            # For the darker colors (after the middle)
            new_lightness = l - (i - middle_index) * lightness_step

        # Ensure new lightness is within the valid range [0, 1]
        new_lightness = max(0, min(new_lightness, 1))

        new_color = colorsys.hls_to_rgb(h, new_lightness, s)
        colors.append(new_color)

    return colors

In [29]:
def get_significance_stars_from_p_value(p_value, number_of_comparisons=3):
    """
    """
    if p_value <= 0.001 / number_of_comparisons:
        return "***"
    elif p_value <= 0.01 / number_of_comparisons:
        return "***"
    elif p_value <= 0.05 / number_of_comparisons:
        return "*"
    else:
        return ""

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [30]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [31]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject',
       ...
       'reward_port', 'box_bottom_left', 'box_bottom_right', 'agent',
       'subject_locations', 'agent_locations', 'subject_thorax_velocity',
       'agent_thorax_velocity', 'subject_thorax_to_reward_port',
       'agent_thorax_to_reward_port'],
      dtype='object', length=153)

In [32]:
TRIAL_LABELS_DF.columns

Index(['tracked_subject', 'box_number', 'sleap_name', 'video_name',
       'current_subject', 'tone_start_frame', 'reward_start_frame',
       'tone_stop_frame', 'condition', 'competition_closeness', 'notes',
       'experiment', 'session_dir', 'all_subjects', 'tone_start_timestamp',
       'tone_stop_timestamp', 'trial_label'],
      dtype='object')

## Merging the labels and LFP spectral

In [33]:
TRIAL_LABELS_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp,trial_label
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,win
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1125,1225,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,lose
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,lose
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,3519,3619,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,win
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",5826740,6026737,lose


In [34]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,reward_port,box_bottom_left,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
1,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 930], [930, 948...","[[32974, 32976], [33201, 33207], [33208, 33211...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[48.01956210909252, 37.264554576109816]","[33.1534452699471, 35.61101869926864]","[62.60288905131385, 35.71974322144712]",None,"[[[27.67316905433137, 31.06298795228111], [27....",NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,"[21.36, 21.36, 21.36, 21.36, 21.36, 21.36, 21....",NaN
2,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",None,"[[[34.36404036584844, 35.615553316175784], [34...",NaN,"[55.4, 52.03, 48.72, 45.53, 42.44, 39.44, 36.5...",NaN,"[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",NaN
3,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.1,"[[[34.36404036584844, 35.615553316175784], [34...","[[[46.68890098287189, 11.495085446056166], [43...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1..."
4,2,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,1.1,...,"[45.31189572048577, 36.38317068198775]","[30.490085836306232, 34.91627159792731]","[60.10744182471764, 34.262899786598894]",None,"[[[46.71964110607613, 36.493379374373234], [44...",NaN,"[5.8, 5.207, 4.64, 4.1, 3.596, 3.127, 2.7, 2.3...",NaN,"[4.633, 4.09, 4.36, 4.656, 4.65, 4.383, 4.35, ...",NaN


In [35]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [36]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)
TRIAL_LABELS_DF["current_subject"] = TRIAL_LABELS_DF["current_subject"].astype(str)

In [37]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1',
       '20230620_114347_standard_comp_

In [38]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230619_115321_standard_comp_

In [39]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["current_subject", "video_name"], how="inner", suffixes=('', '_y'))

In [40]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col], errors="ignore")

In [41]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "_y" in col], errors="ignore")

In [42]:
TRIALS_AND_SPECTRAL_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,reward_port,box_bottom_left,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."


In [43]:
TRIALS_AND_SPECTRAL_DF.columns

Index(['tracked_subject', 'box_number', 'sleap_name', 'video_name',
       'current_subject', 'tone_start_frame', 'reward_start_frame',
       'tone_stop_frame', 'condition', 'competition_closeness',
       ...
       'reward_port', 'box_bottom_left', 'box_bottom_right', 'agent',
       'subject_locations', 'agent_locations', 'subject_thorax_velocity',
       'agent_thorax_velocity', 'subject_thorax_to_reward_port',
       'agent_thorax_to_reward_port'],
      dtype='object', length=158)

In [44]:
raise ValueError()

ValueError: 

# Calculating the relevant timestamps

In [47]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 30 * 20000

In [48]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 20 * 20000

In [49]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

0     526742
1    2926740
2    5226740
3    7026736
4    8126734
Name: baseline_start_timestamp, dtype: int64

In [50]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

0     726742
1    3126740
2    5426740
3    7226736
4    8326734
Name: baseline_stop_timestamp, dtype: int64

In [51]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1126742
1    3526740
2    5826740
3    7626736
4    8726734
Name: tone_start_timestamp, dtype: int64

In [52]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    1326741
1    3726740
2    6026737
3    7826735
4    8926734
Name: tone_stop_timestamp, dtype: int64

- Filtering power

In [53]:
TRIALS_AND_SPECTRAL_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,baseline_start_timestamp,baseline_stop_timestamp
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",526742,726742
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",2926740,3126740
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",5226740,5426740
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",7026736,7226736
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",8126734,8326734


In [54]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [55]:
power_columns

['mPFC_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'vHPC_power_all_frequencies_all_windows',
 'mPFC_power_theta',
 'mPFC_power_gamma',
 'MD_power_theta',
 'MD_power_gamma',
 'LH_power_theta',
 'LH_power_gamma',
 'BLA_power_theta',
 'BLA_power_gamma',
 'vHPC_power_theta',
 'vHPC_power_gamma']

In [56]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_power_all_frequencies
MD_power_all_frequencies
LH_power_all_frequencies
BLA_power_all_frequencies
vHPC_power_all_frequencies
mPFC_power_theta
mPFC_power_gamma
MD_power_theta
MD_power_gamma
LH_power_theta
LH_power_gamma
BLA_power_theta
BLA_power_gamma
vHPC_power_theta
vHPC_power_gamma


In [57]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_power_all_frequencies
MD_power_all_frequencies
LH_power_all_frequencies
BLA_power_all_frequencies
vHPC_power_all_frequencies
mPFC_power_theta
mPFC_power_gamma
MD_power_theta
MD_power_gamma
LH_power_theta
LH_power_gamma
BLA_power_theta
BLA_power_gamma
vHPC_power_theta
vHPC_power_gamma


In [58]:
TRIALS_AND_SPECTRAL_DF["baseline_power_timestamps"]

0       [530000, 540000, 550000, 560000, 570000, 58000...
1       [2930000, 2940000, 2950000, 2960000, 2970000, ...
2       [5230000, 5240000, 5250000, 5260000, 5270000, ...
3       [7030000, 7040000, 7050000, 7060000, 7070000, ...
4       [8130000, 8140000, 8150000, 8160000, 8170000, ...
                              ...                        
1364    [60080000, 60090000, 60100000, 60110000, 60120...
1365    [61580000, 61590000, 61600000, 61610000, 61620...
1366    [63580000, 63590000, 63600000, 63610000, 63620...
1367    [65480000, 65490000, 65500000, 65510000, 65520...
1368    [66480000, 66490000, 66500000, 66510000, 66520...
Name: baseline_power_timestamps, Length: 1369, dtype: object

In [59]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

- Filtering coherence

In [60]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [61]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_theta',
 'BLA_LH_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_MD_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_mPFC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'BLA_vHPC_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_MD_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_mPFC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'LH_vHPC_coherence_gamma',
 'MD_mPFC_coherence_theta',
 'MD_mPFC_coherence_gamma',
 'MD_vHPC_coherence_theta',
 'MD_vHPC_coherence_gamma',


In [62]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

BLA_LH_coherence_all_frequencies
BLA_MD_coherence_all_frequencies
BLA_mPFC_coherence_all_frequencies
BLA_vHPC_coherence_all_frequencies
LH_MD_coherence_all_frequencies
LH_mPFC_coherence_all_frequencies
LH_vHPC_coherence_all_frequencies
MD_mPFC_coherence_all_frequencies
MD_vHPC_coherence_all_frequencies
mPFC_vHPC_coherence_all_frequencies
BLA_LH_coherence_theta
BLA_LH_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_coherence_gamma
BLA_mPFC_coherence_theta
BLA_mPFC_coherence_gamma
BLA_vHPC_coherence_theta
BLA_vHPC_coherence_gamma
LH_MD_coherence_theta
LH_MD_coherence_gamma
LH_mPFC_coherence_theta
LH_mPFC_coherence_gamma
LH_vHPC_coherence_theta
LH_vHPC_coherence_gamma
MD_mPFC_coherence_theta
MD_mPFC_coherence_gamma
MD_vHPC_coherence_theta
MD_vHPC_coherence_gamma
mPFC_vHPC_coherence_theta
mPFC_vHPC_coherence_gamma


In [63]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

BLA_LH_coherence_all_frequencies
BLA_MD_coherence_all_frequencies
BLA_mPFC_coherence_all_frequencies
BLA_vHPC_coherence_all_frequencies
LH_MD_coherence_all_frequencies
LH_mPFC_coherence_all_frequencies
LH_vHPC_coherence_all_frequencies
MD_mPFC_coherence_all_frequencies
MD_vHPC_coherence_all_frequencies
mPFC_vHPC_coherence_all_frequencies
BLA_LH_coherence_theta
BLA_LH_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_coherence_gamma
BLA_mPFC_coherence_theta
BLA_mPFC_coherence_gamma
BLA_vHPC_coherence_theta
BLA_vHPC_coherence_gamma
LH_MD_coherence_theta
LH_MD_coherence_gamma
LH_mPFC_coherence_theta
LH_mPFC_coherence_gamma
LH_vHPC_coherence_theta
LH_vHPC_coherence_gamma
MD_mPFC_coherence_theta
MD_mPFC_coherence_gamma
MD_vHPC_coherence_theta
MD_vHPC_coherence_gamma
mPFC_vHPC_coherence_theta
mPFC_vHPC_coherence_gamma


In [64]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [65]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [66]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows',
 'BLA-to-LH_granger_theta',
 'BLA-to-LH_

In [67]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

BLA_LH_granger_all_frequencies
LH_BLA_granger_all_frequencies
BLA_MD_granger_all_frequencies
MD_BLA_granger_all_frequencies
BLA_mPFC_granger_all_frequencies
mPFC_BLA_granger_all_frequencies
BLA_vHPC_granger_all_frequencies
vHPC_BLA_granger_all_frequencies
LH_MD_granger_all_frequencies
MD_LH_granger_all_frequencies
LH_mPFC_granger_all_frequencies
mPFC_LH_granger_all_frequencies
LH_vHPC_granger_all_frequencies
vHPC_LH_granger_all_frequencies
MD_mPFC_granger_all_frequencies
mPFC_MD_granger_all_frequencies
MD_vHPC_granger_all_frequencies
vHPC_MD_granger_all_frequencies
mPFC_vHPC_granger_all_frequencies
vHPC_mPFC_granger_all_frequencies
BLA-to-LH_granger_theta
BLA-to-LH_granger_gamma
LH-to-BLA_granger_theta
LH-to-BLA_granger_gamma
BLA-to-MD_granger_theta
BLA-to-MD_granger_gamma
MD-to-BLA_granger_theta
MD-to-BLA_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-mPFC_granger_gamma
mPFC-to-BLA_granger_theta
mPFC-to-BLA_granger_gamma
BLA-to-vHPC_granger_theta
BLA-to-vHPC_granger_gamma
vHPC-to-BLA_

In [68]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

BLA_LH_granger_all_frequencies
LH_BLA_granger_all_frequencies
BLA_MD_granger_all_frequencies
MD_BLA_granger_all_frequencies
BLA_mPFC_granger_all_frequencies
mPFC_BLA_granger_all_frequencies
BLA_vHPC_granger_all_frequencies
vHPC_BLA_granger_all_frequencies
LH_MD_granger_all_frequencies
MD_LH_granger_all_frequencies
LH_mPFC_granger_all_frequencies
mPFC_LH_granger_all_frequencies
LH_vHPC_granger_all_frequencies
vHPC_LH_granger_all_frequencies
MD_mPFC_granger_all_frequencies
mPFC_MD_granger_all_frequencies
MD_vHPC_granger_all_frequencies
vHPC_MD_granger_all_frequencies
mPFC_vHPC_granger_all_frequencies
vHPC_mPFC_granger_all_frequencies
BLA-to-LH_granger_theta
BLA-to-LH_granger_gamma
LH-to-BLA_granger_theta
LH-to-BLA_granger_gamma
BLA-to-MD_granger_theta
BLA-to-MD_granger_gamma
MD-to-BLA_granger_theta
MD-to-BLA_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-mPFC_granger_gamma
mPFC-to-BLA_granger_theta
mPFC-to-BLA_granger_gamma
BLA-to-vHPC_granger_theta
BLA-to-vHPC_granger_gamma
vHPC-to-BLA_

In [69]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

# Filtering out the SLEAP posed for during trials

In [70]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col]

In [71]:
sleap_columns

['subject_locations',
 'agent_locations',
 'subject_thorax_velocity',
 'agent_thorax_velocity',
 'subject_thorax_to_reward_port',
 'agent_thorax_to_reward_port']

In [72]:
TRIALS_AND_SPECTRAL_DF["video_timestamps"].iloc[0].shape

(68258,)

In [73]:
TRIALS_AND_SPECTRAL_DF["subject_thorax_to_reward_port"].iloc[0].shape

(68258,)

In [78]:
for col in sorted(sleap_columns):
    updated_item_col = "baseline_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "baseline_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity


In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp]

In [81]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_subject_locations
trial_subject_thorax_to_reward_port
trial_subject_thorax_velocity


In [87]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_subject_thorax_velocity"].apply(lambda x: np.isnan(x).any())]["baseline_subject_locations"]

Series([], Name: baseline_subject_locations, dtype: object)

In [88]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [89]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
start_frame
stop_frame
in_video_subjects
body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent
baseline_start_timestamp
baseline_stop_timestamp
baseline_mPFC_power_all_frequencies
baseline_MD_power_all_frequencies
baseline_LH_power_all_frequencies
baseline_BLA_power_all_frequencies
baseline_vHPC_power_all_frequencies
baseline_mPFC_power_theta
baseline_mPFC_power_gamma
baseline_MD_power_theta
baseline_MD_po

In [90]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))